# Job for data transformation

Este notebook documenta a etapa de transformação de dados da camada Silver, que sucede a extração/ingestão de um relatório institucional de insucesso acadêmico exportado em formato Excel. O arquivo de origem, preservado na camada Bronze com abas por semestre, é lido integralmente e preparado para análise. As transformações incluem a remoção de linhas de rodapé não informativas, o descarte de atributos sem utilidade analítica, a tipagem explícita de identificadores textuais e contagens inteiras, a criação de variáveis derivadas que representem o semestre de oferta e o departamento de origem, bem como a concatenação das diferentes abas em um único conjunto de dados para análise longitudinal.

Do ponto de vista de modelagem, adota-se granularidade por disciplina e semestre, de modo que a combinação de Código e Semestre identifique de forma inequívoca cada registro. A derivação do Departamento a partir do prefixo do Código viabiliza recortes organizacionais consistentes e comparações entre áreas. A padronização de tipos com a explicitação de strings para identificadores e inteiros para contagens evita ambiguidades e reduz o risco de erros durante operações de junção e agregação. Além disso, a imutabilidade por partição temporal é preservada: cada semestre é processado individualmente e, em seguida, incorporado ao conjunto consolidado, o que facilita auditoria e reprodutibilidade.

A governança do processo é apoiada por verificações intermediárias sobre esquemas e contagens, garantindo rastreabilidade das alterações e coerência entre as fontes. Ao final, o resultado é materializado em um arquivo CSV padronizado, `lista-insucesso-processed.csv`, com separador ponto e vírgula, pronto para consumo pelas rotinas de análise e visualização. Reconhece-se, por fim, que a qualidade da análise depende da consistência do arquivo de origem e que investigações causais sobre o insucesso exigiriam variáveis adicionais não contempladas neste relatório; entretanto, a estruturação aqui adotada estabelece uma base sólida para monitoramento perene, comparabilidade entre semestres e expansão futura do escopo analítico.

In [ ]:
# Visão geral programática do Bronze (executar antes das transformações)
import pandas as pd

# Carrega todas as abas
_all = pd.read_excel("../bronze/Relatorio-Lista-Insucesso.xlsx", sheet_name=None)

# Lista de abas e formas
print("Abas:", list(_all.keys()))
for k, df in _all.items():
    print(f"\n--- {k} ---")
    print("shape:", df.shape)
    print("colunas:", list(df.columns))
    print(df.dtypes)

# Exemplo de checagem rápida de nulos por aba
for k, df in _all.items():
    miss = df.isna().sum()
    miss = miss[miss>0]
    if not miss.empty:
        print(f"\nNulos em {k}:\n{miss}")

# Mostra amostra das três primeiras linhas da primeira aba
first_sheet = next(iter(_all))
_all[first_sheet].head(3)


## Imports

In [1]:
import pandas as pd

## Read data

In [2]:
dataframes_all_sheets = pd.read_excel("../bronze/Relatorio-Lista-Insucesso.xlsx", sheet_name=None)

In [3]:
for dataframe in dataframes_all_sheets:
    dataframes_all_sheets[dataframe].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Código                     150 non-null    object 
 1   Nome                       149 non-null    object 
 2   Pólo                       0 non-null      float64
 3   Turmas                     150 non-null    int64  
 4   Discentes                  150 non-null    int64  
 5   Cancelamentos              150 non-null    int64  
 6   Reprovações Média          150 non-null    int64  
 7   Reprovações Nota           150 non-null    int64  
 8   Reprovações Falta          150 non-null    int64  
 9   Reprovações Média e Falta  150 non-null    int64  
 10  Reprovações Nota e Falta   150 non-null    int64  
 11  Trancamentos               150 non-null    int64  
 12  Total Insucesso            150 non-null    int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 

## Transform data

### Create an empty dataframe

In [4]:
finalDataframe = pd.DataFrame()

### Dataframe 2023.2

In [5]:
dataframe1 = dataframes_all_sheets['2023.2']
dataframe1.tail()

,Código,Nome,Pólo,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso
149,FGA0261,TÓPICOS ESPECIAIS EM ELETRONICA,NaN,1,2,2,0,0,0,0,0,0,2
150,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,NaN,1,59,4,1,0,0,0,0,6,11
151,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,NaN,2,94,3,9,0,7,0,0,28,47
152,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,NaN,4,243,8,2,0,0,0,0,1,11
153,Total:,NaN,NaN,327,7369,232,1110,0,316,0,0,547,2205


In [6]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Código                     154 non-null    object 
 1   Nome                       153 non-null    object 
 2   Pólo                       0 non-null      float64
 3   Turmas                     154 non-null    int64  
 4   Discentes                  154 non-null    int64  
 5   Cancelamentos              154 non-null    int64  
 6   Reprovações Média          154 non-null    int64  
 7   Reprovações Nota           154 non-null    int64  
 8   Reprovações Falta          154 non-null    int64  
 9   Reprovações Média e Falta  154 non-null    int64  
 10  Reprovações Nota e Falta   154 non-null    int64  
 11  Trancamentos               154 non-null    int64  
 12  Total Insucesso            154 non-null    int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 

#### Drop last line -> not usable info

In [7]:
dataframe1 = dataframe1.iloc[:-1] 

#### Drop 'Pólo' column

In [8]:
dataframe1 = dataframe1.drop(columns=['Pólo'])

#### Parse columns 'Código' and 'Nome' to string

In [9]:
dataframe1['Código'] = dataframe1['Código'].astype("string")
dataframe1['Nome'] = dataframe1['Nome'].astype("string")

#### Create a column for the semester

In [10]:
dataframe1['Semestre'] = "2023.2"
dataframe1['Semestre'] = dataframe1['Semestre'].astype("string")

#### Create a column for the origin department

In [11]:
dataframe1['Departamento'] = dataframe1['Código'].str[:3]

#### Actual state of the dataframe

In [12]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código                     153 non-null    string
 1   Nome                       153 non-null    string
 2   Turmas                     153 non-null    int64 
 3   Discentes                  153 non-null    int64 
 4   Cancelamentos              153 non-null    int64 
 5   Reprovações Média          153 non-null    int64 
 6   Reprovações Nota           153 non-null    int64 
 7   Reprovações Falta          153 non-null    int64 
 8   Reprovações Média e Falta  153 non-null    int64 
 9   Reprovações Nota e Falta   153 non-null    int64 
 10  Trancamentos               153 non-null    int64 
 11  Total Insucesso            153 non-null    int64 
 12  Semestre                   153 non-null    string
 13  Departamento               153 non-null    string
dtypes: int64(1

In [13]:
dataframe1.tail()

,Código,Nome,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso,Semestre,Departamento
148,JOR0107,TÓPICOS ESPECIAIS EM COMUNICAÇÃO 1,1,1,0,0,0,0,0,0,0,0,2023.2,JOR
149,FGA0261,TÓPICOS ESPECIAIS EM ELETRONICA,1,2,2,0,0,0,0,0,0,2,2023.2,FGA
150,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,1,59,4,1,0,0,0,0,6,11,2023.2,FGA
151,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,2,94,3,9,0,7,0,0,28,47,2023.2,FGA
152,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,4,243,8,2,0,0,0,0,1,11,2023.2,DSC


#### Add transformed data to the final dataframe

In [14]:
finalDataframe = dataframe1.copy()

### Dataframe 2024.1

In [15]:
dataframe1 = dataframes_all_sheets['2024.1']
dataframe1.tail()

,Código,Nome,Pólo,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso
141,FGA0034,TÓPICOS ESPECIAIS EM MATEMÁTICA APLICADA,NaN,1,14,0,0,0,0,0,0,0,0
142,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,NaN,1,29,1,6,0,1,0,0,0,8
143,SER0113,TRABALHO E SOCIABILIDADE,NaN,1,1,0,0,0,0,0,0,0,0
144,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,NaN,1,373,12,9,0,0,0,0,0,21
145,Total:,NaN,NaN,304,6209,299,223,0,97,0,0,0,619


In [16]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Código                     146 non-null    object 
 1   Nome                       145 non-null    object 
 2   Pólo                       0 non-null      float64
 3   Turmas                     146 non-null    int64  
 4   Discentes                  146 non-null    int64  
 5   Cancelamentos              146 non-null    int64  
 6   Reprovações Média          146 non-null    int64  
 7   Reprovações Nota           146 non-null    int64  
 8   Reprovações Falta          146 non-null    int64  
 9   Reprovações Média e Falta  146 non-null    int64  
 10  Reprovações Nota e Falta   146 non-null    int64  
 11  Trancamentos               146 non-null    int64  
 12  Total Insucesso            146 non-null    int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 

#### Drop last line -> not usable info

In [17]:
dataframe1 = dataframe1.iloc[:-1] 

#### Drop 'Pólo' column

In [18]:
dataframe1 = dataframe1.drop(columns=['Pólo'])

#### Parse columns 'Código' and 'Nome' to string

In [19]:
dataframe1['Código'] = dataframe1['Código'].astype("string")
dataframe1['Nome'] = dataframe1['Nome'].astype("string")

#### Create a column for the semester

In [20]:
dataframe1['Semestre'] = "2024.1"
dataframe1['Semestre'] = dataframe1['Semestre'].astype("string")

#### Create a column for the origin department

In [21]:
dataframe1['Departamento'] = dataframe1['Código'].str[:3]

#### Actual state of the dataframe

In [22]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código                     145 non-null    string
 1   Nome                       145 non-null    string
 2   Turmas                     145 non-null    int64 
 3   Discentes                  145 non-null    int64 
 4   Cancelamentos              145 non-null    int64 
 5   Reprovações Média          145 non-null    int64 
 6   Reprovações Nota           145 non-null    int64 
 7   Reprovações Falta          145 non-null    int64 
 8   Reprovações Média e Falta  145 non-null    int64 
 9   Reprovações Nota e Falta   145 non-null    int64 
 10  Trancamentos               145 non-null    int64 
 11  Total Insucesso            145 non-null    int64 
 12  Semestre                   145 non-null    string
 13  Departamento               145 non-null    string
dtypes: int64(1

In [23]:
dataframe1.tail()

,Código,Nome,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso,Semestre,Departamento
140,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,1,43,3,0,0,1,0,0,0,4,2024.1,FGA
141,FGA0034,TÓPICOS ESPECIAIS EM MATEMÁTICA APLICADA,1,14,0,0,0,0,0,0,0,0,2024.1,FGA
142,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,1,29,1,6,0,1,0,0,0,8,2024.1,FGA
143,SER0113,TRABALHO E SOCIABILIDADE,1,1,0,0,0,0,0,0,0,0,2024.1,SER
144,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,1,373,12,9,0,0,0,0,0,21,2024.1,DSC


#### Add transformed data to the final dataframe

In [24]:
finalDataframe = pd.concat([finalDataframe, dataframe1], ignore_index=True)

### Dataframe 2024.2

In [25]:
dataframe1 = dataframes_all_sheets['2024.2']
dataframe1.tail()

,Código,Nome,Pólo,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso
159,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,NaN,1,111,1,37,0,8,0,0,25,71
160,DPG1322,TÓPICOS ESPECIAIS EM SISTEMAS BIOMÉDICOS ELETR...,NaN,1,1,0,0,0,0,0,0,0,0
161,CEM0015,"UNIVERSIDADE, SOCIEDADE E ESTADO",NaN,1,2,0,0,0,1,0,0,0,1
162,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,NaN,1,303,1,14,0,0,0,0,3,18
163,Total:,NaN,NaN,324,7387,206,892,0,263,0,0,523,1884


In [26]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Código                     164 non-null    object 
 1   Nome                       163 non-null    object 
 2   Pólo                       0 non-null      float64
 3   Turmas                     164 non-null    int64  
 4   Discentes                  164 non-null    int64  
 5   Cancelamentos              164 non-null    int64  
 6   Reprovações Média          164 non-null    int64  
 7   Reprovações Nota           164 non-null    int64  
 8   Reprovações Falta          164 non-null    int64  
 9   Reprovações Média e Falta  164 non-null    int64  
 10  Reprovações Nota e Falta   164 non-null    int64  
 11  Trancamentos               164 non-null    int64  
 12  Total Insucesso            164 non-null    int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 

#### Drop last line -> not usable info

In [27]:
dataframe1 = dataframe1.iloc[:-1] 

#### Drop 'Pólo' column

In [28]:
dataframe1 = dataframe1.drop(columns=['Pólo'])

#### Parse columns 'Código' and 'Nome' to string

In [29]:
dataframe1['Código'] = dataframe1['Código'].astype("string")
dataframe1['Nome'] = dataframe1['Nome'].astype("string")

#### Create a column for the semester

In [30]:
dataframe1['Semestre'] = "2024.2"
dataframe1['Semestre'] = dataframe1['Semestre'].astype("string")

#### Create a column for the origin department

In [31]:
dataframe1['Departamento'] = dataframe1['Código'].str[:3]

#### Actual state of the dataframe

In [32]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código                     163 non-null    string
 1   Nome                       163 non-null    string
 2   Turmas                     163 non-null    int64 
 3   Discentes                  163 non-null    int64 
 4   Cancelamentos              163 non-null    int64 
 5   Reprovações Média          163 non-null    int64 
 6   Reprovações Nota           163 non-null    int64 
 7   Reprovações Falta          163 non-null    int64 
 8   Reprovações Média e Falta  163 non-null    int64 
 9   Reprovações Nota e Falta   163 non-null    int64 
 10  Trancamentos               163 non-null    int64 
 11  Total Insucesso            163 non-null    int64 
 12  Semestre                   163 non-null    string
 13  Departamento               163 non-null    string
dtypes: int64(1

In [33]:
dataframe1.tail()

,Código,Nome,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso,Semestre,Departamento
158,FGA0034,TÓPICOS ESPECIAIS EM MATEMÁTICA APLICADA,1,27,1,0,0,0,0,0,0,1,2024.2,FGA
159,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,1,111,1,37,0,8,0,0,25,71,2024.2,FGA
160,DPG1322,TÓPICOS ESPECIAIS EM SISTEMAS BIOMÉDICOS ELETR...,1,1,0,0,0,0,0,0,0,0,2024.2,DPG
161,CEM0015,"UNIVERSIDADE, SOCIEDADE E ESTADO",1,2,0,0,0,1,0,0,0,1,2024.2,CEM
162,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,1,303,1,14,0,0,0,0,3,18,2024.2,DSC


#### Add transformed data to the final dataframe

In [34]:
finalDataframe = pd.concat([finalDataframe, dataframe1], ignore_index=True)

### Dataframe 2025.1

In [35]:
dataframe1 = dataframes_all_sheets['2025.1']
dataframe1.tail()

,Código,Nome,Pólo,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso
145,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,NaN,1,30,0,0,0,0,0,0,1,1
146,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,NaN,1,32,1,0,0,3,0,0,11,15
147,CEM0015,"UNIVERSIDADE, SOCIEDADE E ESTADO",NaN,1,1,0,0,0,1,0,0,0,1
148,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,NaN,1,245,8,11,0,0,0,0,5,24
149,Total:,NaN,NaN,322,6640,202,744,0,210,0,0,578,1734


In [36]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Código                     150 non-null    object 
 1   Nome                       149 non-null    object 
 2   Pólo                       0 non-null      float64
 3   Turmas                     150 non-null    int64  
 4   Discentes                  150 non-null    int64  
 5   Cancelamentos              150 non-null    int64  
 6   Reprovações Média          150 non-null    int64  
 7   Reprovações Nota           150 non-null    int64  
 8   Reprovações Falta          150 non-null    int64  
 9   Reprovações Média e Falta  150 non-null    int64  
 10  Reprovações Nota e Falta   150 non-null    int64  
 11  Trancamentos               150 non-null    int64  
 12  Total Insucesso            150 non-null    int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 

#### Drop last line -> not usable info

In [37]:
dataframe1 = dataframe1.iloc[:-1] 

#### Drop 'Pólo' column

In [38]:
dataframe1 = dataframe1.drop(columns=['Pólo'])

#### Parse columns 'Código' and 'Nome' to string

In [39]:
dataframe1['Código'] = dataframe1['Código'].astype("string")
dataframe1['Nome'] = dataframe1['Nome'].astype("string")

#### Create a column for the semester

In [40]:
dataframe1['Semestre'] = "2025.1"
dataframe1['Semestre'] = dataframe1['Semestre'].astype("string")

#### Create a column for the origin department

In [41]:
dataframe1['Departamento'] = dataframe1['Código'].str[:3]

#### Actual state of the dataframe

In [42]:
dataframe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código                     149 non-null    string
 1   Nome                       149 non-null    string
 2   Turmas                     149 non-null    int64 
 3   Discentes                  149 non-null    int64 
 4   Cancelamentos              149 non-null    int64 
 5   Reprovações Média          149 non-null    int64 
 6   Reprovações Nota           149 non-null    int64 
 7   Reprovações Falta          149 non-null    int64 
 8   Reprovações Média e Falta  149 non-null    int64 
 9   Reprovações Nota e Falta   149 non-null    int64 
 10  Trancamentos               149 non-null    int64 
 11  Total Insucesso            149 non-null    int64 
 12  Semestre                   149 non-null    string
 13  Departamento               149 non-null    string
dtypes: int64(1

In [43]:
dataframe1.tail()

,Código,Nome,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso,Semestre,Departamento
144,FGA0214,TÓPICOS ESPECIAIS EM ENGENHARIA AUTOMOTIVA,1,1,0,0,0,0,0,0,1,1,2025.1,FGA
145,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,1,30,0,0,0,0,0,0,1,1,2025.1,FGA
146,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,1,32,1,0,0,3,0,0,11,15,2025.1,FGA
147,CEM0015,"UNIVERSIDADE, SOCIEDADE E ESTADO",1,1,0,0,0,1,0,0,0,1,2025.1,CEM
148,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,1,245,8,11,0,0,0,0,5,24,2025.1,DSC


#### Add transformed data to the final dataframe

In [44]:
finalDataframe = pd.concat([finalDataframe, dataframe1], ignore_index=True)

### Final dataframe

#### Show statistics of the final data

In [45]:
finalDataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Código                     610 non-null    string
 1   Nome                       610 non-null    string
 2   Turmas                     610 non-null    int64 
 3   Discentes                  610 non-null    int64 
 4   Cancelamentos              610 non-null    int64 
 5   Reprovações Média          610 non-null    int64 
 6   Reprovações Nota           610 non-null    int64 
 7   Reprovações Falta          610 non-null    int64 
 8   Reprovações Média e Falta  610 non-null    int64 
 9   Reprovações Nota e Falta   610 non-null    int64 
 10  Trancamentos               610 non-null    int64 
 11  Total Insucesso            610 non-null    int64 
 12  Semestre                   610 non-null    string
 13  Departamento               610 non-null    string
dtypes: int64(1

In [46]:
finalDataframe.tail()

,Código,Nome,Turmas,Discentes,Cancelamentos,Reprovações Média,Reprovações Nota,Reprovações Falta,Reprovações Média e Falta,Reprovações Nota e Falta,Trancamentos,Total Insucesso,Semestre,Departamento
605,FGA0214,TÓPICOS ESPECIAIS EM ENGENHARIA AUTOMOTIVA,1,1,0,0,0,0,0,0,1,1,2025.1,FGA
606,FGA0054,TÓPICOS ESPECIAIS EM GOVERNANÇA DE TECNOLOGIA ...,1,30,0,0,0,0,0,0,1,1,2025.1,FGA
607,FGA0053,TÓPICOS ESPECIAIS EM PROGRAMAÇÃO,1,32,1,0,0,3,0,0,11,15,2025.1,FGA
608,CEM0015,"UNIVERSIDADE, SOCIEDADE E ESTADO",1,1,0,0,0,1,0,0,0,1,2025.1,CEM
609,DSC0172,VIGILÂNCIA EPIDEMIOLÓGICA PARTICIPATIVA,1,245,8,11,0,0,0,0,5,24,2025.1,DSC


#### Export dataframe to .csv

In [47]:
finalDataframe.to_csv('./lista-insucesso-processed.csv', sep=';',index=False)